In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import time

#import tensorflow.keras as keras
#from keras import backend as K
from sklearn.preprocessing import OneHotEncoder

#import tensorflow as tf
import random

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import math
from IPython.display import display


# Analisis

In [ ]:
totalDir=0
totalFiles=0
for base, dirs, files in os.walk('../input/g2net-gravitational-wave-detection/train'):
    #print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1
        
print('Total number of files',totalFiles,'\n\n')


labels = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
labels.info()



In [ ]:
def get_complete_path(basedir, file_id): 
    #return os.path.join(basedir, file_id[0], file_id[1], file_id[2], file_id+ '.npy' )
    return os.path.join(basedir,file_id[0], file_id[1], file_id[2], file_id+ '.npy' )

In [ ]:
# Revisamos si están balanceadas las clases
print(len(labels))#total de observaciones
labels.target.sum()# total de clase 1

In [ ]:
#leer N ejemplos aleatorios
N=5

options = random.sample(range(0, 150), N)


eventos = [labels['id'].iloc[i] for i in options]
categorias = [labels['target'].iloc[i]for i in options]

print(eventos)

In [ ]:
data_array_list=[]
for evento in eventos:
    path=get_complete_path('../input/g2net-gravitational-wave-detection/train/',evento)
    data_array = np.load(path)
    data_array_list.append(data_array)
    
'''print("\nData summary:\n", data_array)
print("\nData shape:\n", data_array.shape)
'''
df = pd.DataFrame(data_array).T

fig, axs =f, axs = plt.subplots(N,figsize=(15,5*N)) 
for i in range(N):
    for j in range(3):
        
        axs[i].plot(data_array_list[i][j])
        

#df.plot( figsize=(15,4))

print("\n\n\n--------------------------------Las categoría son:",categorias,'---------------------------\n\n')

# Preparación 

In [ ]:


class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        #print(self.labels)
        self.list_IDs = list_IDs
        #print(self.list_IDs)
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        #print(index)
        ID = self.list_IDs[index]
        #print('ID',ID)    
        # Load data and get label
        path = get_complete_path('../input/g2net-gravitational-wave-detection/train', ID)
        #print(path)
        X = torch.from_numpy(np.load(path))
        #print(X)
        y = self.labels[index]
        #print(y)

        return X, y

In [ ]:
def make_trainDL(labels):
    trainingDS = Dataset(labels['id'], labels['target'])
    training_generator = DataLoader(trainingDS,batch_size=1024, shuffle=True)
    return trainingDS , training_generator 
    

In [ ]:
def make_train_data(path,labels):
    data_list=[]
    target_list=[]
    n=1
    for base, dirs, files in os.walk(path):
        for file in files:
            indir=base[:-5]
            n+=1
            file_path = get_complete_path(indir,file)[:-4]
            array = np.load(file_path)
            #array = array.transpose()
            data_list.append(array)
            a=labels.loc[labels['id'] == file[:-4], 'target'].iloc[0]
            target_list.append(a)
    data = np.stack(data_list, axis=0)
    data_l = np.array(target_list)
    data_l = data_l.reshape(data_l.shape[0],-1)
    return data,data_l
            
    

In [ ]:
%%time

msk = np.random.rand(len(labels)) < 0.98
train_data=labels[msk].reset_index()
val_data=labels[~msk].reset_index()
#print(len(train_data),len(val_data))
train, train_dl = make_trainDL(train_data)
val, val_dl = make_trainDL(val_data)


# Para probar el DataLoader

In [ ]:
# Display text and label.
print('\nFirst iteration of data set: ', next(iter(train_dl)), '\n')# Print how many items are in the data set


In [ ]:
#print(train.shape,y_labels.shape)

trainiter = iter(train_dl)
features, labels = next(trainiter)
print(features.shape, labels.shape)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Analisis de frecuencias

from scipy.fft import fft, ifft,fftfreq,rfft,rfftfreq

yf=[]
yf.append(rfft(train[1][0]))
#yf.append(rfft(train[1][1]))
#yf.append(rfft(train[1][2]))

# Number of sample points
print(np.abs(yf[0]),'\n',len(yf[0]))
N = len(train[1][0])
T = 1.0/N

xf = rfftfreq(N, T)

import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.plot(xf[:100],np.abs(yf[0][:100]))

plt.grid()

plt.show()

print('train tiene forma: ',train.shape)
#train=np.transpose(train, (0,2,1))
train.shape

In [ ]:
#onehot_encoder = OneHotEncoder(sparse=False)
#integer_encoded = y_labels.reshape(len(y_labels), 1)
#y_labels = onehot_encoder.fit_transform(integer_encoded)
#print(y_labels)

# dividir en train/validation


X_train, X_val, y_train, y_val = \
    train_test_split(train, y_labels, test_size=0.3, random_state=42)

print('X_train',X_train.shape,'y_train', y_train.shape,'\nX_val',X_val.shape,'y_val',y_val.shape)
ver=X_train[0].shape
print(ver)


X_trainT=torch.from_numpy(X_train)
y_trainT=torch.from_numpy(y_train)
X_valT=torch.from_numpy(X_val)
y_valT=torch.from_numpy(y_val)

train_dl = [[X_trainT,y_trainT]]
val_dl = [[X_valT,y_valT]]



y_trainT.type()
X_trainT.type()

# Definimos un Modelo

In [ ]:
class funcionesBase(nn.Module):
    
    def training_step(self, batch,loss_):
        graf, labels = batch 
        out = self(graf.to(device))
        #print(out.shape,labels.shape)
        #print(out)
        loss = loss_(out.double().to(device), labels.double().to(device)) 
        return loss
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
    def validation_step(self, batch, loss_):
        graf, labels = batch 
        out = self(graf.double().to(device))  
        #labels=labels.long()
        loss = loss_(out.double().to(device), labels.double().to(device)) 
        acc = accuracy(out.to(device), labels.to(device))           
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

def accuracy(outputs, labels):
    preds = torch.round(outputs)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class clasificadorOndasGrav(funcionesBase):
    
    def __init__(self):
        
            super().__init__()   
            self.c1=nn.Conv1d(in_channels=3, out_channels=64, kernel_size=15, stride=4).double()
            self.m1=nn.MaxPool1d(4).double()
            self.c2=nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=2).double()
            self.m2=nn.MaxPool1d(4).double()
            self.c3=nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=2).double()
            self.m3=nn.MaxPool1d(2).double()
            self.c4=nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1).double()
            self.m4=nn.MaxPool1d(2).double()
            self.f=nn.Flatten().double()
            self.l1=nn.Linear(128,128).double()
            self.r1=nn.ReLU().double()
            self.l2=nn.Linear(128,50).double()
            self.r2=nn.ReLU().double()
            self.l3=nn.Linear(50,1).double()
            self.s=nn.Sigmoid().double()
            
    
    def forward(self, xb):
        #self.network = self.network.double()
        out=self.c1(xb.double())
        out=self.m1(out)
        out=self.c2(out)
        out=self.m2(out)
        out=self.c3(out)
        out=self.m3(out)
        out=self.c4(out)
        out=self.m4(out)
        print(out.shape)
        out=self.f(out)
        print(out.shape)
        out=self.l1(out)
        print(out.shape)
        out=self.r1(out)
        out=self.l2(out)
        out=self.r2(out)
        out=self.l3(out)
        out=self.s(out)
        return torch.squeeze(out)

    

In [ ]:
model=clasificadorOndasGrav()
model.to(device)
model

# Para entrenar

In [ ]:
@torch.no_grad()
def evaluate(model, loss_, val_loader):
    model.eval()
    outputs = [model.validation_step(batch, loss_) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, loss_, train_loader,val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for i,batch in enumerate(train_loader):
        
            loss = model.training_step(batch,loss_)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if i%10==0:
                print(".",end="")
            if i%100==0:
                print("")
                result = evaluate(model,loss_, val_loader)
                result['train_loss'] = torch.stack(train_losses).mean().item()
                model.epoch_end(epoch, result)
                history.append(result)
                model.train()
    
    return history

In [ ]:
num_epochs = 3
opt_func = torch.optim.Adam
lr = 0.001

loss=nn.BCELoss() 

history = fit(num_epochs, lr, model, loss, train_dl, val_dl, opt_func)

# Hacer predicción 

In [ ]:
def hacer_predic (model,graf):
    with torch.no_grad():
        y_pred = model(graf)
    return y_pred

In [ ]:
submission = pd.read_csv("../input/g2net-gravitational-wave-detection/sample_submission.csv")
pred_list=[]
for i in range(1):
    evento_a_pred = submission['id'].iloc[i]
    path_pred = get_complete_path('../input/g2net-gravitational-wave-detection/test', evento_a_pred)
    array = np.load(path_pred)
    pred_list.append(array)
    
data = torch.from_numpy(np.stack(pred_list, axis=0))
pred = hacer_predic(model,data)
pred.shape


In [ ]:
submission = pd.read_csv("../input/g2net-gravitational-wave-detection/sample_submission.csv")


submission.to_csv('submission.csv',index=False)

In [ ]:
display(submission)